In [1]:
import requests
import zipfile
import io
import os
import xml.etree.ElementTree as et
import regex as re
from tqdm import tqdm
import json
import xmltodict


In [2]:
links_txt_path='../data/raw/links.txt'

In [10]:
def download_zip_file(url):
    # Send a GET request to the URL
    response = requests.get(url)
    
    # Check if the request was successful (status code 200)
    if response.status_code == 200:
        # Open the zip file using BytesIO
        with io.BytesIO(response.content) as zip_buffer:
            with zipfile.ZipFile(zip_buffer, 'r') as zip_ref:
                # Find the XML file in the zip archive
                xml_file = [file for file in zip_ref.namelist() if file.endswith('.xml')]
                if not xml_file:
                    print("No XML file found in the zip archive.")
                    return None
                # Extract the XML file content
                xml_content = zip_ref.read(xml_file[0])
                # Write into XML file
                xml_file_path=os.path.join("../data/xml_dump",xml_file[0])
                with open(xml_file_path, 'wb') as f: 
                    f.write(xml_content)

                
                return xml_file[0]
    else:
        print("Failed to download file. Status code:", response.status_code)
        return None


In [4]:
# Extract CPC codes from patent documents (just data exploration)
def check_cpc(xml_string):
    # for i, xml_string in tqdm(enumerate(xml_strings)):
        doc = et.fromstring(xml_string)
        cpc_tags = []
        Y02_tags=[]
        # Iterate over all CPC XML tags in the current document
        for elem in doc.iter("classification-cpc-text"):
            cpc_tags.append(elem.text)
        # cpc_map[i] = cpc_tags
        for tag in cpc_tags:
            if tag.startswith("Y02"):
                Y02_tags.append(tag)
        return Y02_tags

In [5]:
# The weekly data dumps from USPTO contain a lot of patent xml documents concatenated in one big xml file
# --> We need to extract the individual documents
# Store each patent xml document as a string in xml_strings
def xml_y02(xml_file):
  xml_strings = []
    # Define SOS sequence that indicates start of new patent
  XML_SOS = re.escape("<?xml") + "*"
  cpc_map = {}
  xml_file_path=os.path.join("../data/xml_dump",xml_file)
  
  count=0
  with open(xml_file_path) as file:
      count=count+1
      xml_string = ""
      i = 0
      cpc_tags=[]
      for line in file:
        if (re.match(XML_SOS, line) != None) & (xml_string != ""):
            i += 1
            cpc_tags=check_cpc(xml_string)
            if cpc_tags!=[]:
              xml_strings.append(xml_string)
            xml_string = ""
            count=count+1
        xml_string += line
  return xml_strings
        

In [6]:
def xml_regex_matches(xml_strings):
    for i,v in enumerate(xml_strings):
        v=v.split('\n')
        for ind,xml_string in enumerate(v):
            xml_string = re.sub("<(b|u|i|(sup)|(sub))>", "", xml_string)
            xml_string = re.sub("<\/(b|u|i|(sup)|(sub))>", "", xml_string)
            xml_string = re.sub(" num=\"[0-9]*\"", "", xml_string)
            xml_string = re.sub(" id=\"((p-)|(ul)|(ul[0-9]*-))[0-9]*\"", "", xml_string)
            xml_string = re.sub(" id=\".*\"", "", xml_string)
            xml_string = re.sub("<li>", "", xml_string)
            xml_string = re.sub("<\/li>", "", xml_string)
            xml_string = re.sub("<ul list-style=\"none\">", "", xml_string)
            xml_string = re.sub("<\/ul>", "", xml_string)
            xml_string = re.sub("<p>", "", xml_string)
            xml_string = re.sub("<\/p>", "", xml_string)
            v[ind]=xml_string

        str=""
        for line in v:
            str=str+line+ '\n'
        xml_strings[i]=str

    return xml_strings


In [7]:
def xml_to_json(xml_strings, file):
  final_json=[]
  for ind,val in enumerate(xml_strings):
      try:
        v=xmltodict.parse(val)['us-patent-grant']
        obj={}
        obj["patent_number"]=v["us-bibliographic-data-grant"]["publication-reference"]["document-id"]["country"]+'-'+v["us-bibliographic-data-grant"]["publication-reference"]["document-id"]["doc-number"]+'-'+v["us-bibliographic-data-grant"]["publication-reference"]["document-id"]["kind"]
        if (type(v["abstract"])==str):
            obj["abstract"]= v["abstract"]
        else: obj["abstract"].get("#text","")
        obj["cpc_id"]=v["us-bibliographic-data-grant"]["us-field-of-classification-search"]['classification-cpc-text']
        final_json.append(obj)
      except:
        pass
  xml_file=os.path.join("../data/json_dump",os.path.splitext(file)[0]+".jsonl")
  with open(xml_file, "w") as outfile:
      for entry in final_json:
          json_str=json.dump(entry,outfile)
          outfile.write('\n')

In [11]:
from datetime import datetime
with open(links_txt_path, 'r') as file:
    for line in file:
        startTime = datetime.now()
        link = line.strip()  
        xml_file = download_zip_file(link)
        # if xml_file!=
        xml_strings= xml_y02(xml_file)
        xml_strings=xml_regex_matches(xml_strings)
        xml_to_json(xml_strings,xml_file)
        print(f"File {xml_file}: {datetime.now()-startTime}")
        # break

Found, skipping
File False: 0:00:11.666532
Found, skipping
File False: 0:00:14.943823


In [ ]:
print(xml_file)

ipg180703.xml
